In [1]:
import ibm_db

In [2]:

dsn_hostname = "dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net" 
dsn_uid = "jlh81393"        # e.g. "abc12345"
dsn_pwd = "2j9q@8ft3wh40tpm"    

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"

In [3]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=jlh81393;PWD=2j9q@8ft3wh40tpm;


In [4]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  BLUDB as user:  jlh81393 on host:  dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net


In [5]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.01.0404
DB_NAME:    BLUDB


In [6]:
import pandas as pd
import ibm_db_dbi

In [7]:
pconn = ibm_db_dbi.Connection(conn)

In [8]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from CRIME"

#retrieve the query results into a pandas dataframe
CRIMES = pd.read_sql(selectQuery, pconn)



In [10]:
%load_ext sql
%sql db2+ibm_db://jlh81393:2j9q%408ft3wh40tpm@dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net:50000/BLUDB

'Connected: jlh81393@BLUDB'

Find the total number of crimes recorded in the Crime table 

In [11]:
%sql select COUNT(id) from CRIME

 * db2+ibm_db://jlh81393:***@dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net:50000/BLUDB
Done.


1
533


Problem 2: Retrieve first 10 rows from the CRIME table

In [16]:
%sql select id, case_number, iucr, primary_type from CRIME limit 10

 * db2+ibm_db://jlh81393:***@dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net:50000/BLUDB
Done.


id,case_number,iucr,primary_type
3512276,HK587712,890,THEFT
3406613,HK456306,820,THEFT
8002131,HT233595,820,THEFT
7903289,HT133522,840,THEFT
10402076,HZ138551,820,THEFT
7732712,HS540106,810,THEFT
10769475,HZ534771,810,THEFT
4494340,HL793243,860,THEFT
3778925,HL149610,810,THEFT
3324217,HK361551,820,THEFT


Problem 3: How many crimes involve an arrest

In [17]:
%sql select COUNT(id) AS total_crimes_involving_arrest from CRIME where arrest = 'TRUE'

 * db2+ibm_db://jlh81393:***@dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net:50000/BLUDB
Done.


total_crimes_involving_arrest
163


Problem 4: Which unique types of crimes have been recorded at a GAS STATION?

In [18]:
%sql select DISTINCT(primary_type) AS gas_station_crimes from CRIME where location_description = 'GAS STATION'

 * db2+ibm_db://jlh81393:***@dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net:50000/BLUDB
Done.


gas_station_crimes
CRIMINAL TRESPASS
NARCOTICS
ROBBERY
THEFT


Problem 5: In the CENUS_DATA table list all community areas whose names start with the letter ‘B’

In [20]:
%sql select community_area_name from CENSUS where community_area_name like 'B%'

 * db2+ibm_db://jlh81393:***@dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net:50000/BLUDB
Done.


community_area_name
Belmont Cragin
Burnside
Brighton Park
Bridgeport
Beverly


Problem 6: List the schools in Community Areas 10 to 15 that are healthy school certified?

In [22]:
%%sql select name_of_school, healthy_school_certified
from PUBLIC_SCHOOL 
where healthy_school_certified = 'Yes' AND community_area_number between 10 and 15

 * db2+ibm_db://jlh81393:***@dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net:50000/BLUDB
Done.


name_of_school,healthy_school_certified
Rufus M Hitch Elementary School,Yes


Problem 7: What is the average school Safety Score?

In [23]:
%sql select AVG(Safety_Score) AS average_school_safety_score from PUBLIC_SCHOOL

 * db2+ibm_db://jlh81393:***@dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net:50000/BLUDB
Done.


average_school_safety_score
49.504873


Problem 8: List the top 5 Community Areas by average College Enrollments [number of students]

In [24]:
%%sql select community_area_name, AVG(college_enrollment) AS average_college_enrollment
from PUBLIC_SCHOOL
group by community_area_name
order by average_college_enrollment desc limit 5

 * db2+ibm_db://jlh81393:***@dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net:50000/BLUDB
Done.


community_area_name,average_college_enrollment
ARCHER HEIGHTS,2411.500000
MONTCLARE,1317.000000
WEST ELSDON,1233.333333
BRIGHTON PARK,1205.875000
BELMONT CRAGIN,1198.833333


Problem 9: Use a sub-query to determine which Community Area has the least value for Safety Score?

In [26]:
%%sql select  community_area_name
from PUBLIC_SCHOOL
where safety_score = (select MIN(safety_score) from  PUBLIC_SCHOOL)

 * db2+ibm_db://jlh81393:***@dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net:50000/BLUDB
Done.


community_area_name
WASHINGTON PARK


Problem 10: Find the Per Capita Income of the Community Area which has a school Safety Score of 1.

In [ ]:
%%sql select  per_capita_income 
from CENSUS
where community_area_number = (select community_area_number from PUBLIC_SCHOOL where safety_score = 1)